This is a test of the translation of the DSS algorithm from Matlab to Python. The Matlab source is taken from `NoiseTools` package that can be found [here](http://audition.ens.fr/adc/NoiseTools/). 

The test consists of running an example from the `NoiseTools` in parallel in Matlab and Python and matching the results.

# Setup

## Imports

In [1]:
from pathlib import Path
import inspect

In [2]:
# Code highlightintg
from pygments import highlight
from pygments.lexers import MatlabLexer, PythonLexer
from pygments.formatters import HtmlFormatter

In [3]:
from IPython.core.display import HTML

In [4]:
# This is a package that allows us to inerface with a Matlab process
from pymatbridge import Matlab

D:\Windows_software\Miniconda3\envs\ding_vs_frank_and_yang\lib\site-packages\IPython\nbformat.py:13: ShimWarning: The `IPython.nbformat` package has been deprecated since IPython 4.0. You should import from nbformat instead.
  "You should import from nbformat instead.", ShimWarning)


In [5]:
import numpy as np
import scipy
from matplotlib import pyplot as plt

## Set up autoreloading of the Python code

In [6]:
%load_ext autoreload
%autoreload 1  # autoreload only the source loaded with %aimport
%aimport dss

## Paths

Change this to wherever you unpacked `NoiseTools` to.

In [7]:
noise_tools_dir = Path('NoiseTools')

In [8]:
noise_tools_examples_dir = noise_tools_dir / 'EXAMPLES' / 'a few old example scripts'
example_1 = noise_tools_examples_dir / 'example1.m'

## Printing code

In [9]:
def print_highlighted_code(code, lexer):
    display(HTML(highlight(code, lexer, HtmlFormatter(full=True, linenos='inline'))))

def print_matlab_script(path):
    with open(path, 'r') as f:
        code = f.read()
    print_highlighted_code(code, MatlabLexer())
    
def print_python_function(fun):
    code = inspect.getsource(fun)
    print_highlighted_code(code, PythonLexer())

## Start Matlab

In [10]:
matlab = Matlab()
matlab.start()

Starting MATLAB on ZMQ socket tcp://127.0.0.1:58791
Send 'exit' command to kill the server
.......MATLAB started and connected!
